In [ ]:
import pyomo.environ as pe
import pyomo.opt as po

In [ ]:
p = 7
c = 5

d = {0: 40, 1: 50, 2: 60, 3: 70}
probability = {0: 0.05, 1: 0.20, 2: 0.40, 3: 0.35}

$$
\begin{alignat*}{2}
f(q) &= \mathbb{E}[p \min\{q, D\}] - cq \\
     &= p \sum_{q=0}^\infty
\end{alignat*}
$$

In [ ]:
solver = po.SolverFactory('gurobi')

In [ ]:
root = pe.ConcreteModel()

root.Omega = pe.Set(initialize=d.keys())

root.p = pe.Param(initialize=p)
root.c = pe.Param(initialize=c)

root.x = pe.Var(domain=pe.NonNegativeIntegers)

def blk_child(block, omega):
    parent = block.model()
    block.d = pe.Param(initialize=d[omega])
    block.probability = pe.Param(initialize=probability[omega])
    block.y = pe.Var(domain=pe.NonNegativeIntegers, bounds=(None, block.d))
    block.con_y_ge_x = pe.Constraint(expr=block.y <= root.x)
    block.revenue = pe.Expression(expr=root.p * block.y)

root.blk_child = pe.Block(root.Omega, rule=blk_child)

def obj(model):
    return sum(child.probability * child.revenue
               for omega, child in model.blk_child.items()) - model.c * model.x

root.obj = pe.Objective(sense=pe.maximize, rule=obj)

solver.solve(root)
print(pe.value(root.x))

In [ ]:
for omega in root.Omega:
    print(pe.value(root.blk_child[omega].y))